**HW 7 Revision**

Price at the Pump

In [27]:
import altair as alt
import pandas as pd

countryCodes = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")
gasolinePrices = pd.read_csv("https://calvin-data304.netlify.app/data/pump_price_for_gasoline_us_per_liter.csv")
gasPricesLong = pd.melt(gasolinePrices, id_vars=['country'], 
                        var_name='Year', value_name='GasPrice')
gasPricesLongSub = pd.merge(gasPricesLong, countryCodes[['name', 'sub-region']], 
                         left_on='country', right_on='name', how='left')

gasPricesLongSub['GasPrice'] = pd.to_numeric(gasPricesLongSub['GasPrice'], errors='coerce')

gasPricesLongSub = gasPricesLongSub.groupby(['sub-region', 'Year'])['GasPrice'].mean().reset_index()

gasPricesLongSub['GasPrice'] = pd.to_numeric(gasPricesLongSub['GasPrice'], errors='coerce')

highlight = alt.selection_point(on='mouseover', fields=['sub-region'], nearest=True)

base_chart = alt.Chart(gasPricesLongSub).mark_line().encode(
    x=alt.X('Year:Q', title="Year", axis=alt.Axis(format='d', title='Year'),
    scale=alt.Scale(domain=[1991,2016])),
    y=alt.Y('GasPrice:Q', title="Average Gas Price By Year"),
    color=alt.Color('sub-region:N', legend=None),
    tooltip=['sub-region:N', 'Year:O', alt.Tooltip('GasPrice:Q', format='$.2f')]
).transform_filter(
    alt.datum.GasPrice != None
)

points = base_chart.mark_circle().encode(
    opacity=alt.value(0)
).add_params(
    highlight
).properties(
    width=500,
    title="Gas Prices Rise and Fall in Parallel Among Sub-Regions"
)

highlight_chart = base_chart.mark_line().encode(
    size=alt.condition(highlight, alt.value(3), alt.value(1))
)

points + highlight_chart

alt.LayerChart(...)